# The __del__ Method - Coding

In [7]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

class Person:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f"Person({self.name})"
    
    def __del__(self):
        print(f"__del__ called for {self}...")
        

p = Person('Alex')
id_p = id(p)
print("Ref count:", ref_count(id_p))

Ref count: 1


In [8]:
p = None

__del__ called for Person(Alex)...


In [9]:
p = Person('Alex')
del p

__del__ called for Person(Alex)...


In [28]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

class Person:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f"Person({self.name})"
    
    def __del__(self):
        print(f"__del__ called for {self}...")
        
    def gen_ex(self):
        raise ValueError("Something went bump...")
        

p = Person('Eric')
id_p = id(p)
print("Ref count:", ref_count(id_p))

Ref count: 1


In [29]:
try: 
    p.gen_ex()
except ValueError as ex:
    error = ex
    print(ex)

Something went bump...


In [30]:
ref_count(id_p)

2

In [31]:
type(error)

ValueError

In [32]:
dir(error)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'args',
 'with_traceback']

In [33]:
dir(error.__traceback__)

['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next']

In [34]:
dir(error.__traceback__.tb_frame)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'f_back',
 'f_builtins',
 'f_code',
 'f_globals',
 'f_lasti',
 'f_lineno',
 'f_locals',
 'f_trace',
 'f_trace_lines',
 'f_trace_opcodes']

In [35]:
type(error.__traceback__.tb_frame.f_locals.items)

builtin_function_or_method

In [36]:
for key, value in error.__traceback__.tb_frame.f_locals.copy().items():
    if isinstance(value, Person):
        print(key, value, id(value))

p Person(Eric) 139743743229760


In [37]:
del p
del error 

__del__ called for Person(Eric)...


In [39]:
class Person:
    def __del__(self):
        raise ValueError("Something get bump...")
        
p = Person()
del p
print("All good, no excepitions interuupted out code.")

Exception ignored in: <function Person.__del__ at 0x7f18a056bb80>
Traceback (most recent call last):
  File "/tmp/ipykernel_9822/1901527688.py", line 3, in __del__
ValueError: Something get bump...


All good, no excepitions interuupted out code.


In [40]:
import sys

sys.stderr, sys.stdout

(<ipykernel.iostream.OutStream at 0x7f18b34e68b0>,
 <ipykernel.iostream.OutStream at 0x7f18b34e6820>)

In [46]:
class ErrToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stderr = sys.stderr
        
        
    def __enter__(self):
        self._file = open(self._fname, "w")
        sys.stderr = self._file
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stderr = self._current_stderr
        if self._file:
            self._file.close()
        return False
    
    
p = Person()

with ErrToFile("err.txt"):
    del p
    print(100)
print(200)
print(300)

100
200
300


In [47]:
with open("err.txt") as f:
    print(f.readlines())

['Exception ignored in: <function Person.__del__ at 0x7f18a056bb80>\n', 'Traceback (most recent call last):\n', '  File "/tmp/ipykernel_9822/1901527688.py", line 3, in __del__\n', 'ValueError: Something get bump...\n']
